# Data Processing and Analysis

In [1]:
# Importing necessary libraries for data analysis, wrangling, and visualization
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
import logging

# Suppress warnings and configure logging
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

In [2]:
# Step 1: Acquire Data
def load_data(file_path):
    """Function to load data from Excel file."""
    try:
        data = pd.read_excel(file_path)
        logging.info("Data successfully loaded!")
        return data
    except Exception as e:
        logging.error(f"Error loading data: {e}")
        return None

# File path to dataset
file_path = r'/content/INX_Future_Inc_Employee_Performance_CDS_Project2_Data_V1.8.xls'
original_data = load_data(file_path)
original_data.head()

,EmpNumber,Age,Gender,EducationBackground,MaritalStatus,EmpDepartment,EmpJobRole,BusinessTravelFrequency,DistanceFromHome,EmpEducationLevel,...,EmpRelationshipSatisfaction,TotalWorkExperienceInYears,TrainingTimesLastYear,EmpWorkLifeBalance,ExperienceYearsAtThisCompany,ExperienceYearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,PerformanceRating
0,E1001000,32,Male,Marketing,Single,Sales,Sales Executive,Travel_Rarely,10,3,...,4,10,2,2,10,7,0,8,No,3
1,E1001006,47,Male,Marketing,Single,Sales,Sales Executive,Travel_Rarely,14,4,...,4,20,2,3,7,7,1,7,No,3
2,E1001007,40,Male,Life Sciences,Married,Sales,Sales Executive,Travel_Frequently,5,4,...,3,20,2,3,18,13,1,12,No,4
3,E1001009,41,Male,Human Resources,Divorced,Human Resources,Manager,Travel_Rarely,10,4,...,2,23,2,2,21,6,12,6,No,3
4,E1001010,60,Male,Marketing,Single,Sales,Sales Executive,Travel_Rarely,16,4,...,4,10,1,3,2,2,2,2,No,3


In [3]:
# Step 2: Analyze Features by Describing Data
def analyze_data(data):
    """Function to analyze features and get an overview of the dataset."""

    if data is not None:
        # Display dataset's features (column names)
        logging.info(f"Features in the dataset:\n{data.columns.values}")

        # Shape of the dataset (rows and columns)
        logging.info(f"Shape of the dataset (rows, columns): {data.shape}")

        # Data types and non-null values in each feature
        logging.info("Data Info:")
        logging.info(data.info())

        # Distribution of categorical features
        logging.info("Categorical feature distribution:")
        logging.info(data.describe(include=['O']))

        # Distribution of numerical features
        logging.info("Numerical feature distribution:")
        logging.info(data.describe())



In [4]:
# Analyze data
analyze_data(original_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   EmpNumber                     1200 non-null   object
 1   Age                           1200 non-null   int64 
 2   Gender                        1200 non-null   object
 3   EducationBackground           1200 non-null   object
 4   MaritalStatus                 1200 non-null   object
 5   EmpDepartment                 1200 non-null   object
 6   EmpJobRole                    1200 non-null   object
 7   BusinessTravelFrequency       1200 non-null   object
 8   DistanceFromHome              1200 non-null   int64 
 9   EmpEducationLevel             1200 non-null   int64 
 10  EmpEnvironmentSatisfaction    1200 non-null   int64 
 11  EmpHourlyRate                 1200 non-null   int64 
 12  EmpJobInvolvement             1200 non-null   int64 
 13  EmpJobLevel       

In [5]:

# Step 3: Data Cleaning and Feature Processing
def clean_data(data):
    """Function to clean data by handling missing values and outliers."""

    # Checking for missing values
    has_nan = data.isna().any().any()
    logging.info(f"Any missing (NaN) values? {has_nan}")

    # Checking for null values (same as NaN check)
    has_null = data.isnull().any().any()
    logging.info(f"Any missing (Null) values? {has_null}")

    if has_nan or has_null:
        # Handling missing values - fill numerical with median, categorical with mode
        for column in data.columns:
            if data[column].dtype == 'object':
                mode_value = data[column].mode()[0]
                data[column].fillna(mode_value, inplace=True)
            else:
                median_value = data[column].median()
                data[column].fillna(median_value, inplace=True)
        logging.info("Missing values handled successfully!")

    # Detect and treat outliers for numerical columns (using IQR)
    for col in data.select_dtypes(include=[np.number]).columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        data[col] = np.where(data[col] > upper_bound, upper_bound, data[col])
        data[col] = np.where(data[col] < lower_bound, lower_bound, data[col])
    logging.info("Outliers handled successfully!")

    return data


In [6]:
# Clean and process the data
cleaned_data = clean_data(original_data.copy())
cleaned_data.head()

,EmpNumber,Age,Gender,EducationBackground,MaritalStatus,EmpDepartment,EmpJobRole,BusinessTravelFrequency,DistanceFromHome,EmpEducationLevel,...,EmpRelationshipSatisfaction,TotalWorkExperienceInYears,TrainingTimesLastYear,EmpWorkLifeBalance,ExperienceYearsAtThisCompany,ExperienceYearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,PerformanceRating
0,E1001000,32.0,Male,Marketing,Single,Sales,Sales Executive,Travel_Rarely,10.0,3.0,...,4.0,10.0,2.0,2.0,10.0,7.0,0.0,8.0,No,3.0
1,E1001006,47.0,Male,Marketing,Single,Sales,Sales Executive,Travel_Rarely,14.0,4.0,...,4.0,20.0,2.0,3.0,7.0,7.0,1.0,7.0,No,3.0
2,E1001007,40.0,Male,Life Sciences,Married,Sales,Sales Executive,Travel_Frequently,5.0,4.0,...,3.0,20.0,2.0,3.0,18.0,13.0,1.0,12.0,No,3.0
3,E1001009,41.0,Male,Human Resources,Divorced,Human Resources,Manager,Travel_Rarely,10.0,4.0,...,2.0,23.0,2.0,2.0,20.5,6.0,7.5,6.0,No,3.0
4,E1001010,60.0,Male,Marketing,Single,Sales,Sales Executive,Travel_Rarely,16.0,4.0,...,4.0,10.0,1.0,3.0,2.0,2.0,2.0,2.0,No,3.0


In [7]:
# Step 4: Feature Encoding and Scaling
def process_features(data):
    """Function to encode categorical features and scale numerical ones."""

    # Label encoding categorical variables
    label_encoders = {}
    for column in data.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column])
        label_encoders[column] = le
        logging.info(f"Encoded categorical feature: {column}")

    # Scaling numerical features using StandardScaler
    scaler = StandardScaler()
    numerical_cols = data.select_dtypes(include=[np.number]).columns
    data[numerical_cols] = scaler.fit_transform(data[numerical_cols])
    logging.info("Numerical features scaled successfully!")

    return data, label_encoders, scaler

In [8]:
# Encode and scale the data
processed_data, encoders, scaler = process_features(cleaned_data.copy())

In [9]:
# Step 5: Save Processed Data
def save_processed_data(data, file_name="processed_data.csv"):
    """Function to save the cleaned and processed data for further modeling."""
    try:
        data.to_csv(file_name, index=False)
        logging.info(f"Processed data saved as {file_name}")
    except Exception as e:
        logging.error(f"Error saving data: {e}")

In [10]:
# Save processed data for later modeling
save_processed_data(processed_data)

In [11]:
# Final check of the processed data
processed_data.head()

,EmpNumber,Age,Gender,EducationBackground,MaritalStatus,EmpDepartment,EmpJobRole,BusinessTravelFrequency,DistanceFromHome,EmpEducationLevel,...,EmpRelationshipSatisfaction,TotalWorkExperienceInYears,TrainingTimesLastYear,EmpWorkLifeBalance,ExperienceYearsAtThisCompany,ExperienceYearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,PerformanceRating
0,-1.730608,-0.541458,0.809427,-0.179459,1.236181,1.052350,0.833426,0.596508,0.102061,0.103000,...,1.185833,-0.157922,-0.688233,-1.064491,0.587144,0.771213,-0.778981,1.121693,-0.417335,0.0
1,-1.727721,1.109888,0.809427,-0.179459,1.236181,1.052350,0.833426,0.596508,0.591464,1.061145,...,1.185833,1.204942,-0.688233,0.365956,0.032653,0.771213,-0.374491,0.834907,-0.417335,0.0
2,-1.724835,0.339260,0.809427,-0.943112,-0.132285,1.052350,0.833426,-0.876351,-0.509693,1.061145,...,0.255768,1.204942,-0.688233,0.365956,2.065785,2.465670,-0.374491,2.268839,-0.417335,0.0
3,-1.721948,0.449349,0.809427,-1.706766,-1.500751,-0.126754,-0.218658,0.596508,0.102061,1.061145,...,-0.674297,1.613801,-0.688233,-1.064491,2.527861,0.488804,2.254698,0.548120,-0.417335,0.0
4,-1.719061,2.541054,0.809427,-0.179459,1.236181,1.052350,0.833426,0.596508,0.836165,1.061145,...,1.185833,-0.157922,-1.663298,0.365956,-0.891498,-0.640834,0.030000,-0.599025,-0.417335,0.0


## Components of the Notebook:
1. **Data Acquisition:** Loads the dataset using pandas and logs whether the  operation was successful.

2. **Data Analysis:** Provides an overview of the dataset, including shape, column names, and distributions of features (both categorical and numerical).

3. **Data Cleaning** Handles missing values and outliers by filling numerical columns with median values and categorical ones with mode. Outliers are capped using IQR.

4. **Feature Encoding and Scaling:** Encodes categorical variables with LabelEncoder and scales numerical features using StandardScaler.

5. **Saving Processed Data:** Saves the processed data to a CSV file for further analysis and modeling.